### Création d’un index simple

- Si on effectue souvent des recherches par email, on peut ajouter un index sur cette colonne :

```sql
CREATE INDEX idx_utilisateurs_email ON utilisateurs(email);
```

#### Test de performance avec EXPLAIN ANALYZE

```sql
EXPLAIN ANALYZE SELECT * FROM utilisateurs WHERE email = 'bob@example.com';
```

Filter: ((email)::text = 'bob@example.com'::text)  
Rows Removed by Filter: 1  
Planning Time: 0.272 ms  
Execution Time: 0.026 ms  

### Index sur plusieurs colonnes

- Si une requête filtre souvent sur ```nom``` et ```email```, on peut créer un index composite :

```sql
CREATE INDEX idx_utilisateurs_nom_email ON utilisateurs(nom, email);
```

Attention : Un index composite n’est utile que si la requête filtre sur la première colonne (ici nom), sinon il ne sera pas utilisé.

### Index GIN pour les recherches textuelles avancées

- Pour rechercher des mots dans un texte, PostgreSQL propose l’index ```GIN``` :

```sql
CREATE INDEX idx_utilisateurs_nom_gin ON utilisateurs USING gin(to_tsvector('french', nom));
```

- Puis, une recherche optimisée :

```sql
SELECT * FROM utilisateurs WHERE to_tsvector('french', nom) @@ to_tsquery('Martin');
```

### Transactions et performances

Les transactions garantissent que plusieurs opérations s’exécutent ensemble ou pas du tout (Atomicité).

```sql
BEGIN;
UPDATE utilisateurs SET age = age + 1 WHERE id = 1;
INSERT INTO commandes (utilisateur_id, produit, prix) VALUES (1, 'Clavier', 49.99);
COMMIT;
```

Si une erreur survient, on peut annuler avec :

```sql
ROLLBACK;
```

### Optimisation avec ```BATCH INSERT```

Au lieu d’insérer ligne par ligne (lent) :

```sql
INSERT INTO commandes (utilisateur_id, produit, prix) 
VALUES (1, 'Clavier', 49.99);
```


Il vaut mieux insérer plusieurs lignes en une seule requête :

```sql
INSERT INTO commandes (utilisateur_id, produit, prix) 
VALUES (1, 'Clavier', 49.99), (2, 'Souris', 29.99), (3, 'Écran', 199.99);
```

= Gains de performance 🚀 : Moins d’allers-retours entre le client et le serveur.

### Eviter les SELECT *

✅ Bonne pratique :

```sql
SELECT nom, email FROM utilisateurs;
```

### Choisir le bon type de données

- ```VARCHAR(255)``` ❌ (inutilement grand)
- ```VARCHAR(50)``` ✅ (plus rapide en RAM)
- ```TEXT``` ✅ (si taille variable et texte long)
- ```BOOLEAN``` ✅ (plutôt que ```INTEGER pour stocker ```0/1```)
- ```TIMESTAMP WITH TIME ZONE``` ✅ (évite les problèmes de fuseaux horaires)

### Partitionnement des tables

- Si une table devient très grande (millions de lignes), on peut la partitionner :

```sql
CREATE TABLE commandes_2024 PARTITION OF commandes 
FOR VALUES FROM ('2024-01-01') TO ('2024-12-31');
```

= Chaque année aura sa propre partition, améliorant les performances.

### Indexation Avancée : Accélérer les Recherches

#### Index B-Tree (par défaut)

Accélère les recherches par égalité et ordre (```=```, ```<```, ```>```, ```BETWEEN```)

```sql
EXPLAIN ANALYZE SELECT * FROM produits WHERE nom = 'MacBook';
```

= Sans index, PostgreSQL fait un Sequential Scan (lecture complète de la table). Avec un B-Tree index, il fait un Index Scan, bien plus rapide.

#### Index Hash (Optimisé pour égalité)

Si on ne fait que des recherches exactes (```=```)

```sql
CREATE INDEX idx_email_hash ON utilisateurs USING hash(email);
```

= Plus rapide que B-Tree pour = mais ne gère pas ```<```, ```>```, ```BETWEEN```.

#### Index GIN (Optimisé pour JSON et Texte)

Recherche rapide dans les colonnes JSON ou TEXT

```sql
CREATE INDEX idx_json_gin ON produits USING gin(data_jsonb);
```

Accélère les recherches sur les champs JSON :

```sql
SELECT * FROM produits WHERE data_jsonb @> '{"marque": "Apple"}';
```

#### Index Full-Text Search (Recherche Texte Rapide)

Recherche sur des textes longs (titre, description), utiliser un index GIN avec tsvector.

```sql
CREATE INDEX idx_search ON articles USING gin(to_tsvector('french', contenu));
```

Accélère:

```sql
SELECT * FROM articles WHERE to_tsvector('french', contenu) @@ to_tsquery('ordinateur & portable');
```

#### Index BRIN (Optimisé pour les tables très grandes)

Utilisation : Si les données sont insérées de manière ordonnée (ex : dates)

```sql
CREATE INDEX idx_date_brin ON commandes USING brin(date_commande);
```

Gain énorme pour des tables contenant plusieurs millions de lignes.

### Partitionnement des Tables : Gérer de Gros Volumes

Si une table devient trop grande (plusieurs millions de lignes), PostgreSQL ralentit. La solution ? Partitionner la table.

#### Création d’une table partitionnée (par date)

```sql
CREATE TABLE ventes (
    id SERIAL,
    date_vente DATE NOT NULL,
    produit TEXT,
    montant NUMERIC
) PARTITION BY RANGE (date_vente);
```

Créer des partitions mensuelles :

```sql
CREATE TABLE ventes_2024_01 PARTITION OF ventes 
FOR VALUES FROM ('2024-01-01') TO ('2024-02-01');
```

= PostgreSQL enregistre automatiquement les nouvelles ventes dans la bonne partition !

✅ Accélération des requêtes :

```sql
EXPLAIN ANALYZE SELECT * FROM ventes WHERE date_vente BETWEEN '2024-01-01' AND '2024-01-31';
```

👉 PostgreSQL ne parcourt que la partition concernée !

### Analyse et Optimisation des Requêtes

Pour améliorer les performances, il faut comprendre comment PostgreSQL exécute les requêtes avec ```EXPLAIN ANALYZE```.

#### Détection des requêtes lentes

```sql
EXPLAIN ANALYZE SELECT * FROM commandes WHERE client_id = 123;
```

📌 Résultat possible :

```sql
Seq Scan on commandes (cost=0.00..125.00 rows=500 width=32)
```

🔴 Problème : PostgreSQL fait un "Seq Scan" (lecture complète de la table).  
✅ Solution : Ajouter un index :

```sql
CREATE INDEX idx_client_id ON commandes(client_id);
```

Après l’indexation, le plan devrait afficher "Index Scan", beaucoup plus rapide !

#### Cache et Optimisation de la Mémoire

📌 PostgreSQL utilise un cache interne.

- Vérifier la taille du cache

```sql
SHOW shared_buffers;
```

= Par défaut sur Mac, c’est souvent 128MB, trop peu pour des bases volumineuses.

- Augmenter la mémoire allouée à PostgreSQL

Dans ```postgresql.`conf``` :

shared_buffers = 512MB  
work_mem = 64MB  
effective_cache_size = 2GB  


= Gains : Moins d’accès disque, plus de requêtes exécutées en RAM !

|Optimisation|Utilisation|Impact|
|:---|:---|:---|
|Index B-Tree|Recherches ```=``` ```<``` ```>```|⚡ Rapide|
|Index GIN|Recherches JSON et Texte|⚡ Très rapide|
|Partitionnement|Tables énormes (millions de lignes)|🔥 Essentiel|
|EXPLAIN ANALYZE|Détecter les requêtes lentes|🔍 Analyse|
|Mémoire (```shared_buffers```)|Eviter les accès disque|⚡ Boost|